In [65]:
from src.helpers import *
from sklearn.pipeline import Pipeline
from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier
from datetime import datetime
import time

In [66]:
# Reload the saved scaled data
X_train = pd.read_csv('data/train/scaled_X_train.csv', parse_dates=True, index_col='unix')
y_train = pd.read_csv('data/train/y_train.csv', parse_dates=True, index_col='unix')
X_test = pd.read_csv('data/test/scaled_X_test.csv', parse_dates=True, index_col='unix')
y_test = pd.read_csv('data/test/y_test.csv', parse_dates=True, index_col='unix')
X_val = pd.read_csv('data/val/scaled_X_val.csv', parse_dates=True, index_col='unix')
y_val = pd.read_csv('data/val/y_val.csv', parse_dates=True, index_col='unix')

In [75]:
start = time.time()
for i in range(1, 1000):
    continue
end = time.time()
duration = '{0:.5f}'.format(end - start)
print(f'{duration} seconds')

0.00006 seconds


In [53]:
# Calculate the weights for the imbalanced classes
y = pd.concat([y_train, y_val, y_test])
weights = cwts(y.values.flatten())


In [54]:
rf = RandomForestClassifier(n_jobs=-1, class_weight=weights)

In [55]:
# Define the various feature selection methods
vif = VIFTransform(dropna=True)
boruta = BorutaPy(rf, n_estimators='auto', verbose=2)

In [56]:
# Define data pipelines
pipelines = [
    'none',
    Pipeline([('vif', vif)])
]

In [57]:
pipe_1 = Pipeline([('vif', vif), ('boruta', boruta)])

In [63]:
filepath = './tensorboard/feature_selection/run'
time = datetime.now().strftime('%m-%d-%Y-%H:%M:%S')
for key in pipe_1.named_steps.keys():
    filepath += f'_{key}_'
filepath += time
filepath

'./tensorboard/feature_selection/run_vif__boruta_12-31-2023-17:12:45'

In [23]:
y_train.values.ravel()

array([0, 0, 0, ..., 0, 0, 0])

In [25]:
y_val.values.ravel()

array([0, 0, 0, ..., 0, 0, 0])

In [16]:
# X_train = pipe_1.fit_transform(X_train, y_train.values.ravel())

Calculating VIF Factors


/home/boshlike/.local/share/virtualenvs/final_assignment_code-eTELY0YF/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


Calculating VIF Factors - Complete
Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	33
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	33
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	33
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	33
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	33
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	33
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	33
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	6
Rejected: 	27
Iteration: 	9 / 100
Confirmed: 	2
Tentative: 	4
Rejected: 	27
Iteration: 	10 / 100
Confirmed: 	2
Tentative: 	4
Rejected: 	27
Iteration: 	11 / 100
Confirmed: 	2
Tentative: 	4
Rejected: 	27
Iteration: 	12 / 100
Confirmed: 	2
Tentative: 	3
Rejected: 	28
Iteration: 	13 / 100
Confirmed: 	2
Tentative: 	3
Rejected: 	28
Iteration: 	14 / 100
Confirmed: 	2
Tentative: 	3
Rejected: 	28
Iteration: 	15 / 100
Confirmed: 	2
Tentative: 	3
Rejected: 	28
Iteration: 	16 / 100
Confirme

In [19]:
pipe_1.support_
X_train

AttributeError: 'Pipeline' object has no attribute 'support_'